# **The Optimal Diet Notebook**
This is the offical Google Colab notebook for the YouTube video you just watched by **Roman Kenric** about the cheapest diet that meets all of the daily essential nutrients for a human.

In this notebook you will have access to all of the data as well as the instructions to verify the work and get the exact amount of each food item for you based on the amount of calories and macro nutrients you want in a day.

---
## **Instructions**
Everything in this notebok is setup for you to just press play and go. This means all you need to do is hover over the code box and press the play button that pops up.

Now every block of code is meant to be run in order, and each code block will be labeled at the top in green next to a # symbol.

Some code blocks have specific insturctions where you need to do something like uploading the data files you downloaded with this project.

**Step 1: Importing Python Packages**

In [8]:
# Code Block 1: loading the Python packages
import pandas as pd
import numpy as np
from scipy.optimize import linprog
from google.colab import files
from IPython.display import HTML

**Step 2: Upload the CSV files downloaded from the video description**

In [9]:
# Code Block 2: Uploading Datasets
# You will need to upload the csv files
# that were downloaded to your computer
files.upload()

Saving constraints.csv to constraints.csv
Saving nutrients.csv to nutrients.csv


{'constraints.csv': b'Nutrient,Min,Max\r\nProtein,150,250\r\nFat,40,70\r\nCarbs,160,310\r\nFiber,15,120\r\nVitamin A,700,2800\r\nVitamin E,14,800\r\nVitamin K,100,1000\r\nVitamin C,75,1800\r\nThiamin (B1),1.1,400\r\nRiboflavin (B2),1.1,400\r\nNiacin (B3),14,800\r\nPantothenic Acid (B5),5,500\r\nVitamin B6,1.3,40\r\nBiotin (B7),30,100\r\nFolate (B9),400,800\r\nVitamin B12,2.4,300\r\nCalcium,800,2000\r\nPhosphorus,700,3000\r\nMagnesium,400,900\r\nPotassium,3000,5000\r\nSodium,1500,4000\r\nChloride,300,4000\r\nIron,8,45\r\nZinc,11,40\r\nIodine,150,900\r\nSelenium,50,400\r\nCopper,900,10000\r\nManganese,2,11\r\nChromium,30,600\r\nMolybdenum,45,1000\r\nOmega-3 Fatty Acids,0.25,3\r\nOmega-6 Fatty Acids,11,28\r\nCalories,1900,2600',
 'nutrients.csv': b'Food,Protein (g/g),Fat (g/g),Carbohydrates (g/g),Fiber (g/g),Vitamin A (\xce\xbcg/g),Vitamin E (mg/g),Vitamin K (\xce\xbcg/g),Vitamin C (mg/g),Thiamin (B1) (mg/g),Riboflavin (B2) (mg/g),Niacin (B3) (mg/g),Pantothenic Acid (B5) (mg/g),Vitamin B6

**Step 3: Code to create the datasets**

In [10]:
# Code Block 3: Prepping The Data
nutrients_df = pd.read_csv('nutrients.csv')
constraints_df = pd.read_csv('constraints.csv')

**Step 4: Select the amount of calories, protein, fat, and carbs you want**

This is the most important part where you will have to change the numbers based on what you want. Please follow the instructions carefully.

**Key Rule:** The max has to be either equal or greater. For example, if min_carbs_percentage is 40, then max_carbs_percentage needs to be 40 or greater.


In [11]:
# Code Block 4: Selecting the macro nutrients and calories

## Macro nutrient percentages
## Note: this is in percent, so the only acceptable answers ar 1 to 100.
## Also, the sum should be 100. For example, min_protein_percentage + min_fat_percentage
## + min_carbs_percentage = 100
## The same goes for the Max

#### EXAMPLE START ####
#       min_calories = 1700
#       max_calories = 2200
#       min_protein_percentage = 30 ------- (Can either be 1 - 100)
#       max_protein_percentage = 30
#       min_fat_percentage = 20
#       max_fat_percentage = 20
#       min_carbs_percentage = 50
#       max_carbs_percentage = 50

#### EXAMPLE END ####


## Calories: min_calories at least 1300 and max_calories must be equal to or greater than min_calories
min_calories = 1900
max_calories = 2600
min_protein_percentage = 31.58
max_protein_percentage = 38.46
min_fat_percentage = 18.95
max_fat_percentage = 24.23
min_carbs_percentage = 33.68
max_carbs_percentage = 47.69

## **Step 5: Linear Program Code**
All of the code blocks in here can be run at the same time by just pressing the run (play) button. Just hover over here and press the button and then go to **step 6**

In [12]:
# Cal
constraints_df.at[32,'Min'] = min_calories
constraints_df.at[32,'Max'] = max_calories
# Protein
constraints_df.at[0,'Min'] = round((min_calories * (min_protein_percentage/100))/4)
constraints_df.at[0,'Max'] = round((max_calories * (max_protein_percentage/100))/4)
#Fat
constraints_df.at[1,'Min'] = round((min_calories * (min_fat_percentage/100))/9)
constraints_df.at[1,'Max'] = round((max_calories * (max_fat_percentage/100))/9)
#carbs
constraints_df.at[2,'Min'] = round((min_calories * (min_carbs_percentage/100))/4)
constraints_df.at[2,'Max'] = round((max_calories * (max_carbs_percentage/100))/4)

In [13]:
#################### Optimization Program ####################
#############################################################
# Extract coefficients for the objective function (cost per gram)
c = nutrients_df['Cost Per Gram'].values

# Define columns to exclude
exclude_columns = ['Food', 'Cost Per Gram']

# Select all columns except the ones to exclude
nutrient_columns = [col for col in nutrients_df.columns if col not in exclude_columns]

# Extract the values of the selected columns and transpose them
nutrients = nutrients_df[nutrient_columns].T.values

# Define the bounds for each nutrient based on df2
min_intake = constraints_df['Min'].values
max_intake = constraints_df['Max'].values

# Combine constraints into a single inequality system
A = []
b = []

# Add constraints for minimum intake (-A_ub * x <= -b)
A.extend(-nutrients)   # Negative because we're flipping the inequality
b.extend(-min_intake) # Negative because we're flipping the inequality

# Add constraints for maximum intake (A_ub * x <= b)
A.extend(nutrients)
b.extend(max_intake)

A = np.array(A)
b = np.array(b)

# Define bounds for each food item (non-negative quantities)
food_bounds = [(0, None) for _ in range(len(nutrients_df))]

# Solve the linear programming problem
result = linprog(c=c, A_ub=A, b_ub=b, bounds=food_bounds, method='highs')

In [14]:
# Getting the quantities for the foods
food_quantities = {}
for food, quantity in zip(nutrients_df['Food'], result.x):
    food_quantities[food] = quantity

food_quantities_df = pd.DataFrame(list(food_quantities.items()), columns=["Food", "Quantity (Grams)"])
food_quantities_df["Quantity (Grams)"] = food_quantities_df["Quantity (Grams)"].round(2)

In [15]:
# Getting Total Nutrients Information
# Assuming result.x is a numpy array or something similar
opt_quant = [float(q) for q in result.x]

# Calculate the total nutrient content for each column
total_nutrients = {}
for nutrient in nutrient_columns:
    total_nutrients[nutrient] = np.dot(nutrients_df[nutrient], opt_quant)

total_nutrients_df = pd.DataFrame(list(total_nutrients.items()), columns=["Nutrient", "Nutrient Level From Solution"])
total_nutrients_df["Nutrient Level From Solution"] = total_nutrients_df["Nutrient Level From Solution"].round().astype(int)

total_nutrients_df['Min'] = constraints_df['Min']
total_nutrients_df['Max'] = constraints_df['Max']

total_nutrients_df["Max"] = total_nutrients_df["Max"].round().astype(int)
total_nutrients_df["Min"] = total_nutrients_df["Min"].round().astype(int)

total_nutrients_df["In Range"] = total_nutrients_df.apply(
    lambda row: "Yes" if row["Min"] <= row["Nutrient Level From Solution"] <= row["Max"] else "No", axis=1)

In [16]:
def display_side_by_side(*dfs):
    html = '<div style="display:flex">'
    for df in dfs:
        html += '<div style="margin-right: 20px">'  # Add space between tables
        html += df.to_html()
        html += '</div>'
    html += '</div>'
    display(HTML(html))

## **Step 6: Final Output**
Run these code blocks to get your anserws

In [19]:
print('\n Final Output:')
print('----------------')
print(f'\n Total Daily Cost: ${round(result.fun,2)}')
print('\n')
print(display_side_by_side(total_nutrients_df,food_quantities_df))


 Final Output:
----------------

 Total Daily Cost: $3.17




None


In [18]:
# Check if the optimization was successful
if result.success:
    print("Optimal solution found!")
    print("\nQuantities of each food (grams):")
    for food, quantity in zip(nutrients_df['Food'], result.x):
        print(f"{food}: {quantity:.2f} grams")
    print(f"\nTotal cost: ${result.fun:.2f}")
else:
    print("Optimization failed:", result.message)

Optimal solution found!

Quantities of each food (grams):
Lentils: 37.93 grams
Eggs: 0.00 grams
Sardines: 19.56 grams
Chicken Breast: 101.02 grams
Chickpeas: 252.85 grams
Black Beans: 131.79 grams
Oats: 0.00 grams
Brown Rice: 0.00 grams
Whole Grain Bread: 21.84 grams
Quinoa: 0.00 grams
Broccoli: 54.28 grams
Carrots: 53.14 grams
Yams: 0.00 grams
Spinach: 0.00 grams
Beets: 0.00 grams
Red Onions: 0.00 grams
Canned Corn: 0.00 grams
Green Cabbage: 30.13 grams
Bananas: 0.00 grams
Apples: 0.00 grams
Oranges: 0.00 grams
Greek Yogurt: 0.00 grams
Pumkin Seeds: 0.00 grams
Peanuts: 0.00 grams
Milk: 333.61 grams
Cottage Cheese: 0.00 grams
Tofu: 3.97 grams
Mayonnaise: 0.00 grams
Mustard: 0.00 grams
Cinnamon: 0.00 grams
Salt: 3.04 grams
Paprika: 0.00 grams
Sunflower Seeds: 75.88 grams
Limes: 0.00 grams
Canola Oil: 0.00 grams
Tuna: 0.00 grams

Total cost: $3.17
